# 导入模块

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 
from pandas.tseries.offsets import MonthEnd
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean


In [61]:
pd.set_option('display.max_columns', None)
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 连接数据库

In [62]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [63]:
sql = '''
select 
om.create_time, ybt.order_id, ybt.add_time, ybt.end_time, om.order_number, ybt.`status`, om.has_actual, ybt.money, ybt.assign_user_name, ybt.completion_amount
, om.order_finish_date, om.end_date end_date_o, ybt.type,  om.status status_s, tprm.rent_received, tprm.purchase_amount, tod.new_actual_money, too.real_pay_money
from db_rent.ya_buyout_task ybt
left join db_digua_business.t_order om on om.id=ybt.order_id
left join (select order_id, sum(real_pay_money) real_pay_money from db_digua_business.t_order_out where status!=1 group by order_id) too on too.order_id=ybt.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=ybt.order_id
left join db_digua_business.t_order_details tod on tod.order_id=ybt.order_id
-- where date_format(ybt.end_time, '%Y-%m')='2025-03'
'''
df = query(sql)
df = df[df.money>300]

In [64]:
sql_gd = '''
select tmwo.order_id
,tmwo.audit_status, tmwo.out_method, yat.add_time add_time_a
from db_digua_business.t_merchant_work_order tmwo
left join db_rent.ya_agent_task yat on yat.order_id=tmwo.order_id
'''
df_gd = query(sql_gd)
# df_gd = df_gd.sort_values('add_time_a').groupby('order_id').head(1)
df_gd.shape

(32665, 4)

In [65]:
sql_top = '''
select order_id, describes, create_time from db_digua_business.t_order_personnel
'''
df_top = query(sql_top)
df_top = df_top.sort_values(by='create_time', ascending=False).groupby('order_id').head(1)

In [66]:
sql_yofl = '''
select order_id, content, type type_y, time, user_name 跟进人 from db_rent.ya_order_follow_log
'''
df_yofl = query(sql_yofl)
df_yofl = df_yofl.sort_values(by='time').groupby('order_id').head(1)


In [67]:
sql_ck = '''
select order_number, add_time add_time_c from db_rent.ya_inventory_output
'''
df_ck = query(sql_ck)
df_ck = df_ck.sort_values(by='add_time_c', ascending=False).groupby('order_number').head(1)
df_ck.loc[:, '出库月份'] = df_ck.add_time_c.dt.strftime('%Y-%m')

In [68]:
sql_pos = '''
select 
tprm.order_number, tprm.order_id, tprm.all_rental, tod.new_actual_money, tprm.rent_received, tprm.total_received, om.has_actual, tprm.purchase_amount, date_format(tol.go_express_date, '%Y-%m') 出库月份
, tprm.search_time, om.merchant_name, tod.sku_attributes, pa.type, om.order_finish_date, om.update_time, too.discount_money, too.real_pay_money, yatl.id, too.status status_t, om.create_time
, tprm.all_deposit
from db_digua_business.t_postlease_receivables_monitoring tprm 
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 物流表
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
-- 活动表
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 买断表
left join (select * from db_digua_business.t_order_out where status!=1) too on too.order_id=om.id
-- 法诉表
left join db_rent.ya_agent_task_lawsuit yatl on yatl.order_id=om.id
where tprm.purchase_amount>1
'''
df_pos = query(sql_pos)
df_pos = clean.drop_merchant(df_pos)
df_pos = clean.drop_rejected_merchant(df_pos)
df_pos = df_pos[df_pos.type!=4]
df_pos = df_pos[df_pos.id.isna()]
# df_pos = df_pos[df_pos.status_t!=1]

In [69]:
df_pos.loc[:, '已买断'] = np.where(df_pos.has_actual==1, 1, 0)
df_pos.loc[:, '应付买断尾款'] = np.where(df_pos.has_actual==1, df_pos.new_actual_money-df_pos.all_rental, 0)
df_pos.loc[:, '实付买断尾款'] = np.where(df_pos.status_t!=1, df_pos.real_pay_money, 0)
df_pos.loc[:, '买断尾款'] = df_pos.new_actual_money-df_pos.all_rental
df_pos.loc[:, '优惠金额'] = np.where(df_pos.status_t!=1, df_pos.discount_money, 0)
df_pos_g2 = df_pos.groupby('search_time').agg({'order_number': 'count', '已买断': 'sum', '应付买断尾款': 'sum', '实付买断尾款': 'sum', '优惠金额': 'sum'}).rename(columns={'order_number': '出库数'})
df_pos_g2.index.name='下单月份'

In [70]:
df_pos.loc[:, '买断'] = np.where(df_pos.has_actual==1, 1, 0)
df_pos.loc[:, 'order_finish_date'] = np.where(df_pos.order_finish_date.isna(), df_pos.update_time, df_pos.order_finish_date)
df_pos.loc[:, '买断月份'] = np.where(df_pos.has_actual==1, df_pos.order_finish_date.dt.strftime('%Y-%m'), np.nan)
df_pos.loc[:, '买断日期'] = np.where(df_pos.has_actual==1, df_pos.order_finish_date.dt.strftime('%Y-%m-%d'), np.nan)
df_pos.loc[:, '下单日期'] = df_pos.create_time.dt.strftime('%Y-%m-%d')
# 将那些无法转换的值标记为 NaT
df_pos["下单日期"]=pd.to_datetime(df_pos["下单日期"],errors="coerce")
df_pos.loc[:, '出库月份'] = np.where(df_pos.出库月份.isna(), df_pos.search_time, df_pos.出库月份)
df_pos.loc[:, '买断金额'] = np.where(df_pos.has_actual==1, df_pos.new_actual_money, 0)
# df_pos_g = df_pos.groupby('出库月份').agg({'order_number': 'nunique', '买断': 'sum', 'new_actual_money': 'sum', 'total_received': 'sum', 'purchase_amount': 'sum'}).rename(columns={'order_number': '出库数', 'new_actual_money': '应收租金', 'total_received': '已收租金', 'purchase_amount': '采购金额'})
# df_pos_g.loc[:, '应收毛利率'] = ((df_pos_g.应收租金-df_pos_g.采购金额)/df_pos_g.采购金额).map(lambda x: format(x, '.2%'))
# df_pos_g.loc[:, '已收毛利率'] = ((df_pos_g.已收租金-df_pos_g.采购金额)/df_pos_g.采购金额).map(lambda x: format(x, '.2%'))
# df_pos_g.loc[:, '出库买断率_订单'] = (df_pos_g.买断/df_pos_g.出库数).map(lambda x: format(x, '.2%'))
# df_pos_g.loc[:, '出库买断率_金额'] = (df_pos_g.已收租金/df_pos_g.应收租金).map(lambda x: format(x, '.2%'))
df_pos_g = df_pos.groupby('出库月份').agg({'order_number': 'nunique', '买断': 'sum', 'new_actual_money': 'sum', '买断金额': 'sum'}).rename(columns={'order_number': '出库数', 'new_actual_money': '出库金额'})
df_pos_g.loc[:, '出库买断率_订单'] = (df_pos_g.买断/df_pos_g.出库数).map(lambda x: format(x, '.2%'))
df_pos_g.loc[:, '出库买断率_金额'] = (df_pos_g.买断金额/df_pos_g.出库金额).map(lambda x: format(x, '.2%'))
# df_pos_g2 = df_pos.groupby('买断月份').agg({'买断': 'sum', '买断金额': 'sum'}).rename(columns={'new_actual_money': '出库金额'})
# df_pos_g2

In [71]:
df_pos_md = df_pos[df_pos.买断==1]
df_pos_md.loc[:, 'day'] = (pd.to_datetime(df_pos_md.买断日期)-pd.to_datetime(df_pos_md.下单日期)).dt.days
df_pos_md_g = df_pos_md.groupby('买断月份').agg({'买断': 'sum', 'day': 'sum'})
df_pos_md_g.loc[:, '平均买断时长'] = np.round((df_pos_md_g.day/df_pos_md_g.买断), 2)


# 促买断数据

In [72]:
t_date = datetime.today().strftime('%Y%m%d')
df = df.merge(df_ck, on='order_number', how='left')
df.loc[:, '出库月份'] = np.where(df.出库月份.isna(), df.create_time.dt.strftime('%Y-%m'), df.出库月份)

In [73]:
df.loc[:, 'add_date'] = df.add_time.dt.strftime('%Y-%m')
df.loc[:, 'end_date'] = df.end_time.dt.strftime('%Y-%m')
df.loc[:, 'create_date'] = df.create_time.dt.strftime('%Y-%m')
df['real_pay_money'] = df.real_pay_money.fillna(0)
# 促买断订单
df_group = df.groupby('add_date').agg(促买断数量=('order_id', 'nunique'), 促买断金额=('money', 'sum'), 采购金额=('purchase_amount', 'sum'))
df_group2 = df.groupby('出库月份').agg(促买断数量=('order_id', 'nunique'), 促买断金额=('money', 'sum'), 采购金额=('purchase_amount', 'sum'))

# 促买断已完成订单
df_finish = df[(df.status==3)] # &(df.has_actual==1)
df_finish_group = df_finish.groupby('add_date').agg(促买断已完成数量=('order_id', 'nunique'), 促买断已完成金额=('money', 'sum'), 买断价=('new_actual_money', 'sum'), 已收租金1=('rent_received', 'sum'), 已收租金2=('real_pay_money', 'sum'))
df_finish_group.loc[:, '优惠金额'] = df_finish_group.买断价-(df_finish_group.已收租金1+df_finish_group.已收租金2)
df_group_all = df_group.merge(df_finish_group[['促买断已完成数量', '促买断已完成金额', '优惠金额']], on='add_date')
df_group_all.index.name='添加日期'
df_group_all.loc[:, '买断完成率'] = (df_group_all.促买断已完成数量/df_group_all.促买断数量).map(lambda x: format(x, '.2%'))
df_group_all.loc[:, '买断金额完成率'] = (df_group_all.促买断已完成金额/df_group_all.促买断金额).map(lambda x: format(x, '.2%'))
df_group_all

df_finish_group2 = df_finish.groupby('出库月份').agg(促买断已完成数量=('order_id', 'nunique'), 促买断已完成金额=('money', 'sum'), 买断价=('new_actual_money', 'sum'), 已收租金1=('rent_received', 'sum'), 已收租金2=('real_pay_money', 'sum'))
df_finish_group2.loc[:, '优惠金额'] = df_finish_group2.买断价-(df_finish_group2.已收租金1+df_finish_group2.已收租金2)
df_group_all2 = df_group2.merge(df_finish_group2[['促买断已完成数量', '促买断已完成金额', '优惠金额']], on='出库月份')
df_group_all2.index.name='出库月份'
df_group_all2.loc[:, '买断完成率'] = (df_group_all2.促买断已完成数量/df_group_all2.促买断数量).map(lambda x: format(x, '.2%'))
df_group_all2.loc[:, '买断金额完成率'] = (df_group_all2.促买断已完成金额/df_group_all2.促买断金额).map(lambda x: format(x, '.2%'))
# df_group_all2
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/买断数据_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_group_all.to_excel(writer, sheet_name='买断数据')
#     df_group_all2.to_excel(writer, sheet_name='买断数据_出库月份')
#     df_pos_g.to_excel(writer, sheet_name='历史总体买断数据_出库月份')
#     df_pos_g2.to_excel(writer, sheet_name='历史总体买断数据_下单月份')
#     df_pos_md_g.to_excel(writer, sheet_name='平均买断时长')

In [74]:
# 促买断订单
df_group = df.groupby('create_date').agg(促买断数量=('order_number', 'count'))
# 促买断已完成订单
df_finish = df[(df.status==3)] # &(df.has_actual==1)
df_finish_group = df_finish.groupby('create_date').agg(促买断已完成数量=('order_number', 'count'))
df_group_all = df_group.merge(df_finish_group, on='create_date')
# df_group_all

# 已取消的已买断完成

In [75]:

df_new = df.merge(df_top, on='order_id', how='left').merge(df_yofl, on='order_id', how='left')
df_new.loc[:, 'order_finish_month'] = df_new.order_finish_date.dt.strftime('%Y-%m')
# df_new = df_new[df_new.跟进人==df_new.assign_user_name]
df_new = df_new[((df_new.type==0)&(df_new.status==4)&(df_new.status_s==8)&(df_new.has_actual==1)&(df_new.order_finish_month!='2025-03')&(df_new.type_y==5))][['order_number', 'order_finish_date', 'describes', 'content', 'assign_user_name', '跟进人', 'type_y']].rename(columns={'order_number': '订单号', 'order_finish_date': '订单完成时间', 'describes': '描述', 'content': '内容','assign_user_name': '分配人'})
# df_new.to_excel(f'F:/需求/廖志彧需求/已取消的买断完成订单_{t_date}.xlsx', index=False)
df_new.shape

(274, 7)

# 当月发布当月买断

In [76]:
df_now = df.merge(df_gd, on='order_id', how='left').merge(df_ck, on='order_number', how='left')
# df_now = df_now[df_now.add_time_a>df_now.add_time]
df_now = df_now.sort_values('add_time', ascending=False).groupby('order_id').head(1)
df_now.shape

(9551, 28)

In [77]:
# df_now_month = df_now[(df_now.add_date=='2025-03')]
# df_end_month = df_now[df_now.end_date=='2025-03']
# # df_now_month.loc[:, '提前买断'] = np.where((df_now_month.out_method==2)&(df_now_month.audit_status==2), 1, 0)
# # df_now_month.loc[:, '已逾期'] = np.where(((df_now_month.end_time+timedelta(days=2)>=df_now_month.add_time_a))&(df_now_month.add_time_a>df_now_month.add_time)&(df_now_month.status==4), 1, 0)
# df_now_month.loc[:, '当月买断'] = np.where((df_now_month.end_date=='2025-03')&((df_now_month.status==3)), 1, 0)
# df_now_month.loc[:, '非当月买断'] = np.where((df_now_month.end_date>'2025-03')&((df_now_month.status==3)), 1, 0)
# df_end_month['real_pay_money'] = df_end_month.real_pay_money.fillna(0)
# df_end_month.loc[:, '截至月份已收租金'] = df_end_month.rent_received + df_end_month.real_pay_money

# # df_end_month.loc[:, '提前买断'] = np.where((df_end_month.out_method==2)&(df_end_month.audit_status==2), 1, 0)
# df_end_month.loc[:, '提前买断'] = np.where((df_end_month.type==1)&(df_end_month.status==3), 1, 0)
# df_end_month.loc[:, '已逾期'] = np.where(((df_end_month.end_time+timedelta(days=30)>=df_end_month.add_time_a))&(df_end_month.add_time_a>df_end_month.add_time)&(df_end_month.status==4), 1, 0)

# # 当月买断
# df_now_month_g = df_now_month.groupby('add_date').agg({'order_id': 'nunique', '当月买断': 'sum'}).rename(columns={'order_id': '总发布订单数'})
# df_now_month_g.index.name = '当前月份'
# df_now_month_g.loc[:, '当月买断率'] = (df_now_month_g.当月买断/df_now_month_g.总发布订单数).map(lambda x: format(x, '.2%'))
# df_now_month_g
# # 非当月买断
# df_now_month_g2 = df_now_month.groupby(['add_date']).agg({'order_id': 'nunique', '非当月买断': 'sum'}).rename(columns={'order_id': '总发布订单数'})
# df_now_month_g2.index.name = '当前月份'
# df_now_month_g2.loc[:, '非当月买断率'] = (df_now_month_g2.非当月买断/df_now_month_g2.总发布订单数).map(lambda x: format(x, '.2%'))
# df_now_month_g2
# # 排除提前买断、逾期
# df_end_month_new = df_end_month[(df_end_month.提前买断==0)]
# df_end_month_new = df_end_month_new[df_end_month_new.已逾期==0]
# df_end_month_new.loc[:, '总买断'] = np.where(df_end_month_new.status==3, 1, 0)
# df_end_month_new_g = df_end_month_new.groupby(['出库月份','add_date']).agg({'order_id': 'count', '总买断': 'sum'}).rename(columns={'order_id': '发布订单数'})
# df_end_month_new_g.index.name = '当前月份'
# df_end_month_new_g.loc[:, '总买断率'] = (df_end_month_new_g.总买断/df_end_month_new_g.发布订单数).map(lambda x: format(x, '.2%'))
# df_end_month_new_g
# # 毛利率
# df_now_month_ml = df_end_month[df_end_month.status==3]
# df_now_month_ml_g = df_now_month_ml.groupby('add_date').agg({'new_actual_money': 'sum', '截至月份已收租金': 'sum', 'purchase_amount': 'sum'}).rename(columns={'new_actual_money': '截至月份应收租金', 'purchase_amount': '采购金额'})
# df_now_month_ml_g.index.name = '当前月份'
# df_now_month_ml_g.loc[:, '应收毛利率'] = ((df_now_month_ml_g.截至月份应收租金 - df_now_month_ml_g.采购金额)/df_now_month_ml_g.采购金额).map(lambda x: format(x, '.2%'))
# df_now_month_ml_g.loc[:, '实收毛利率'] = ((df_now_month_ml_g.截至月份已收租金 - df_now_month_ml_g.采购金额)/df_now_month_ml_g.采购金额).map(lambda x: format(x, '.2%'))
# df_now_month_ml_g

# # with pd.ExcelWriter(f'F:/需求/廖志彧需求/促买断数据_{t_date}.xlsx', engine='xlsxwriter') as writer:
# #     df_now_month_g.to_excel(writer, sheet_name='当月买断')
# #     df_now_month_g2.to_excel(writer, sheet_name='非当月买断')
# #     df_end_month_new_g.to_excel(writer, sheet_name='总体买断')
# #     df_now_month_ml_g.to_excel(writer, sheet_name='毛利率')


# 下单月份买断出库数

In [82]:
df_pos.loc[:, '下单日期'] = pd.to_datetime(df_pos.下单日期)
df_pos.loc[:, '下单月份'] = df_pos.create_time.dt.strftime('%Y-%m')
df_pos_new = df_pos[df_pos.下单日期>='2023-01-01']
# lis = []
# for n in range(1, 16):
#     df_pos_new[f'mob{n}'] = np.where((df_pos_new["下单日期"].dt.day == 31)|(df_pos_new["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df_pos_new["下单日期"].dt.month.isin([4,6,9,11]))&(df_pos_new["下单日期"].dt.day == 30)), df_pos_new['下单日期'] + MonthEnd(n), df_pos_new['下单日期'] + MonthEnd(n+1))
#     df_pos_new['MOB'] = f'mob{n}'
#     lis.append(df_pos_new)

# df_pos_new_mob = df_pos_new[['下单日期', 'order_id', 'mob1','mob2','mob3','mob4','mob5','mob6','mob7','mob8','mob9','mob10','mob11','mob12','mob13','mob14','mob15']]
# df_pos_new_g = df_pos_new.groupby('下单日期').agg({'order_number': 'nunique', '已买断': 'sum', '应付买断尾款': 'sum', '实付买断尾款': 'sum', '买断尾款': 'sum', '优惠金额': 'sum', 'all_rental': 'mean', 'all_deposit': 'mean'}).rename(columns={'order_number': '出库数', 'all_rental': '总租金均值', 'all_deposit': '总押金均值'})

# df_pos_new_g.loc[:, '买断率_订单'] = (df_pos_new_g.已买断/df_pos_new_g.出库数).map(lambda x: format(x, '.2%'))
# df_pos_new_g.loc[:, '买断率_金额'] = (df_pos_new_g.买断尾款/df_pos_new_g.实付买断尾款).map(lambda x: format(x, '.2%'))
# df_pos_new_g.loc[:, '租售比'] = (df_pos_new_g.总租金均值/df_pos_new_g.总押金均值).map(lambda x: format(x, '.2%'))
# df_pos_merge = df_pos_new_g.merge(df_pos_new_mob, on='下单日期', how='left')
# df_pos_merge.drop_duplicates(subset=['下单日期'], inplace=True)
# df_pos_merge
# df_pos_new[(df_pos_new.下单日期=='2023-01-03')&(df_pos_new.已买断==1)][['order_number', '已买断', '应付买断尾款', '实付买断尾款', '优惠金额']]

In [85]:
df_pos_new.shape

(29871, 32)

In [96]:
sql_buyout = '''
select
ybt.order_id, ybt.status
from db_rent.ya_buyout_task ybt
left join db_digua_business.t_order om on om.id=ybt.order_id
where date_format(om.create_time, '%Y-%m-%d') >= '2023-01-01' and ybt.status=3 and om.has_actual=1
'''
df_buyout = query(sql_buyout)
# 方式一
df_pos_new_merge = df_pos_new[['order_id', '已买断']].merge(df_buyout, on='order_id', how='left')
df_pos_new_merge.loc[:, '买断方式'] = np.where((df_pos_new_merge.已买断==1)&(df_pos_new_merge.status.isna()), '自然买断', np.where((df_pos_new_merge.status==3), '促买断', '未买断'))
# 方式二
df_buyout_list = df_buyout.order_id.to_list()
df_pos_new.loc[:, '买断方式'] = np.where(df_pos_new.order_id.isin(df_buyout_list), '促买断', np.where(df_pos_new.已买断==1, '自然买断', '未买断'))

In [99]:
df_pos_new[df_pos_new.买断方式=='自然买断'].shape

(2895, 33)